In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '.././')
import asym_funcs as af
import os

In [2]:
datadir = './output/'

In [3]:
# Modify function to allow latitudinal variations in heat capacity

In [4]:
def simple_model (nspace=800, nts=1000, mysolar=0., mycw=9.8, myA=138.6, dur=100, myFb=5., myB=2.1, forc_xt = 0.):
    """
    Run the simple model globally. Can vary parameters, resolution and run length as input to this function
    """                                                                                                                                                                                                                                  
    # Physical parameters
    S1 = 338.;     # insolation seasonal dependence (W m^-2)
    A  = myA #193  # OLR when T = 0 (W m^-2)
    B  = myB #2.1  # OLR temperature dependence (W m^-2 K^-1)
    cw = mycw      # ocean mixed layer heat capacity (W yr m^-2 K^-1)
    S0 = 420.      # insolation at equator  (W m^-2)
    S2 = 240.      # insolation spatial dependence (W m^-2)
    a0 = 0.7       # ice-free co-albedo at equator
    a2 = 0.1       # ice=free co-albedo spatial dependence
    Fb = myFb #4   # heat flux from ocean below (W m^-2)
  
    # Time stepping parameters
    #dur          # of years for the whole run
    n  = nspace;  # of evenly spaced latitudinal gridboxes (pole to pole)
    nt = nts;     # of timesteps per year (approx lower limit of stability) 
    dt = 1/nt;
    ty = np.arange(dt/2,1+dt/2,dt)
    print(cw.shape)
    print("Arguments: nx = {0}, nt = {1}, duration = {2} years".format(nspace,nts,dur))
    print("Parameters:  A = {}, Fb = {}, B = {}".format(A, Fb, B))
    print("Forcing: solar {}, additional {}".format(np.max(np.abs(mysolar==0.))>0, np.max(np.abs(forc_xt))>0))
    
    #Spatial Grid -------------------------------------------------------------
    dx = 2.0/n    #grid box width
    x = np.arange(-1+dx/2,1+dx/2,dx) #native grid
    lat = np.arcsin(x)*180./np.pi
  
    ##Seasonal forcing (WE15 eq.3)
    if np.all(mysolar==0.): # if an insolation field is not provided, use the idealized function
        S = (np.tile(S0-S2*x**2,[nt,1])- np.tile(S1*np.cos(2*np.pi*ty),[n,1]).T*np.tile(x,[nt,1])); 
    else:
        S = mysolar.T
    S = np.vstack((S,S[0,:]))
    
    Amod = np.zeros([nt,n])
    if np.all(forc_xt == 0.):
        Amod[:,:] = A
    else:
        Amod[:,:] = - forc_xt[:,:].T + A
    Amod = np.vstack((Amod,Amod[0,:]))
      
    alpha = a0-a2*x**2      # open water albedo

    #Set up output arrays, saving all timesteps of all years
    Tfin  = np.zeros([dur,nt,n])
   
    #Initial conditions ------------------------------------------------------
    T = 7.5+20*(1-2*x**2);
    
    #Loop over Years ---------------------------------------------------------
    for years in range(0,dur):
        #Loop within One Year-------------------------------------------------
        for i in range(0,int(nt)):
            
            #forcing
            T = T + (dt/cw[:])*(alpha*S[i+1] - (Amod[i+1]+B*T)+Fb)                
            Tfin[years,i,:] = T
                   
    T_all = xr.DataArray(Tfin,dims=('year','time','lat'),coords = {'year':np.arange(1,dur+1,1), 'time':ty, 'lat':lat}).to_dataset(name='T')
    
    return T_all

In [5]:
sol = xr.open_dataset('forcingdata/forcing_TOAinsolation_n800.nc').S.values
heatcap = xr.open_dataset('forcingdata/zonal_variation_in_cw_n800.nc')

In [6]:
ds = simple_model(mysolar=sol, mycw = heatcap.c.values)
myname = 'ODEn800_TOAinsolation_wgtheatcap'
ds['names'] = myname
ds = ds.set_coords('names')
ds = ds.isel(year=slice(-20,None)).mean(dim='year')
ds.to_netcdf(datadir+myname+'.nc')

(800,)
Arguments: nx = 800, nt = 1000, duration = 100 years
Parameters:  A = 138.6, Fb = 5.0, B = 2.1
Forcing: solar True, additional False
